 # Location Analysis Sample

This demo illustrates adding a spatial dimension to an existing
information system. The existing system did not contain any explicit
location (spatial) data. However, the existing system did contain implicit
location data in the form of addresses.  By spatially enabling the existing
database, the user expands the business analysis capabilities of the system.


A jupyter notebook version of
https://www.ibm.com/blogs/cloud-archive/2015/08/location-location-location/
  
  
In this scenario, a small company (MYCO) has two offices, but business has been
growing and there are now customers across the country. Many of the customers have
expressed a preference to meet company representatives in person. The company owners
want to explore where to open a new office.

Some of the questions in MYCO company owners want to answer are:

    We already have some ideas where to open a new office. How can we find out which of these potential locations can serve the most customers?
    How can we reach the customers with the highest business volume?
    Are there other locations that should be considered?

Spatial analysis functions can help find the answers

On Db2 Warehouse on Cloud the geospatial data used to bring this example to life can be found in the SAMPLE schema. It contains data about customers in the GEO_CUSTOMER table and county data in the GEO_COUNTIES table. You can use the Tables menu to view the structure and browse the content of these tables.

## Install Python Libraries

In [ ]:
 !pip install folium ibm_db pandas geopandas

## Setup - Create a database 

In [ ]:
# Adjust the settings as necessary. For a local database the log file changes are necessary because the defaults
# are not sufficient to work with the data and queries.

# Switch to warehouse configuration - column organized default and some other settings are different
#!db2set DB2_WORKLOAD=ANALYTICS

#!db2 -v "create database BLUDB using codeset UTF-8 territory US collate using identity pagesize 32 k"

# Make sure log files are big enough if on prem or local env out of the box
#!db2 update db cfg using logfilsiz 4000
#!db2 update db cfg using logprimary 10
#!db2 update db cfg using logsecondary 50

## Setup - Enable Spatial Analytics 

In [ ]:
!db2 -v "connect to BLUDB"
!db2 -v "CALL SYSPROC.SYSINSTALLOBJECTS('GEO', 'C', NULL, NULL)"

## Create and Insert data 

This notebook assumes the data files for the tables are unzipped.
The zipped files can be found in the "spatial/data" directory.

In [ ]:
## If the insert into GEO_CUSTOMER fails with
## SQL0901N  The SQL statement or command failed because of a database system 
## error. (Reason "Max internal handles".)  SQLSTATE=58004
## Then set registry variable to bypass LOB handle issue on INSERT:
!db2set DB2_TCG_DEFAULT_OPTIONS="set may_release_lob_handle on"
## After changing the registry variable the instance needs to be recycled.
!db2stop force
!db2start

# Now connect and create tables and load data.
# The load_data.sql script assumes the data files are unzipped in "../data/"
# as this Jupyter notebook and script.
# The files are zipped and located in SAMPLE_PATH/spatial/data.
!db2 -v "connect to BLUDB"
!db2 -tvf load_data.sql

## Connect IBM_DB (Python) client with database

In [ ]:
# !db2start

# Note: a connection can be local via CLP or via the Python. This notebook uses both intermingled.
import ibm_db
from ibm_db import connect

# Local connection
connection = connect('DSN=BLUDB;', '', '')

# For a remote connection using TCP - instance must be configured properly to listen to TCP connections
# via DB2COMM and DBM CFG SVCENAME/SSL_SVCENAME

#connection = connect('DATABASE=BANKDEMO;'
#                     'HOSTNAME=127.0.0.1;'
#                     'PORT=50000;'
#                     'PROTOCOL=SOCKETS;'
#                     'UID=db2inst1;'
#                     'PWD=password;', '', '')

## Import Essential Libraries 

In [ ]:
import pandas as pd
import folium
from folium import plugins
from shapely import wkt
import geopandas as gpd

# Begin of scenario

The current offices are in Pittsburgh and Atlanta. The locations are inserted with the datatype ST_POINT and, as in the following SQL statements, the spatial reference system with the id 1005 is used, which refers to the most popular coordinate system WGS1984:

In [ ]:
create_table_offices="CREATE TABLE IF NOT EXISTS offices ( \
  id integer not null primary key, \
  name     varchar(30) not null, \
  location st_point not null, \
  contact  varchar(128), \
  status   integer not null default 0 \
)"
ibm_db.exec_immediate(connection,create_table_offices)

# Two office locations
insert_table_offices="delete from offices; \
       insert into offices \
       values (1, 'Pittsburgh', st_point(-79.5835, 40.2623, 1005), 'Dan Smith',0), \
              (2, 'Atlanta', st_point(-84.2324, 33.4518, 1005), 'Jane Miller',0)"
ibm_db.exec_immediate(connection,insert_table_offices)


The customer base has been growing in the west and southwest, so the MYCO owners consider opening an office in Austin, Dallas, Las Vegas, or San Jose. The question now is: Which of these locations is the best suited location as far as customers in its vicinity are concerned.

To answer this question, the potential locations are added to the OFFICE table with status ‘1’ to indicate ‘planned’ status:

In [ ]:
# Add possible office locations
insert_table_offices=" insert into offices \
       values (11, 'Las Vegas', st_point(-115.1739, 36.1215, 1005), 'tbd', 1), \
              (12, 'San Jose', st_point(-121.924 , 37.3591, 1005), 'tbd', 1), \
              (13, 'Austin', st_point(-97.76, 30.3208, 1005), 'tbd', 1), \
              (14, 'Dallas', st_point(-96.974, 32.6406, 1005), 'tbd', 1)"
ibm_db.exec_immediate(connection,insert_table_offices)

## Queries and visualization

It is mainly the customers with high business volume that are interested in a local contact. If we define 200 miles as the maximum distance for this scenario, then the first step is to find out how many customers with a given business volume are located closer than 200 miles to existing and planned offices.

Spatial Analytics supports two options to find distances, one is using the st_distance function and the other is a combination of st_buffer and st_intersects.
A quick performance check shows that using the first option

    st_intersects(st_buffer(off.location,200,'STATUTE MILE'), cust.shape) = 1

results in a runtime of about 7mins.

Using the second option 

    st_distance(off.location, cust.shape, 'STATUTE MILE') < 200

reduces the time to about 13s on the same data, same system, same environment.

The second option is recommended when using a unit, here the ‘STATUTE MILE’, so we use the second option:

In [ ]:
# Could also create this query using a view. That is define new view that performs the select and then query data from the view.
query="select o1.name as office_name, o2.count_customers as customers, st_y(o1.location) as office_lat, st_x(o1.location) as office_lon \
       from offices o1, \
            (select off.id as office_id, off.name as office_name, count(*) as count_customers \
             from offices off, geo_customer cust \
             where cust.insurance_value > 200000 and \
                   st_distance(off.location, cust.shape, 'STATUTE MILE') < 200 \
             group by off.id, off.name \
             order by count_customers) as o2 \
       where o1.id = o2.office_id order by customers desc"

# Takes about 13s on SMP in VM.
stmt = ibm_db.exec_immediate(connection, query)
result = ibm_db.fetch_both(stmt)
OFFICE_NAME = []
NUM_CUSTOMERS = []
OFFICE_LAT = []
OFFICE_LON = []

# Read result into arrays.
while( result ):
    OFFICE_NAME.append(result[0])
    NUM_CUSTOMERS.append(result[1])
    OFFICE_LAT.append(result[2])
    OFFICE_LON.append(result[3])
    result = ibm_db.fetch_both(stmt)

In [ ]:
numRows = len(OFFICE_NAME)
txt = "Office num results = {}"
print(txt.format(numRows))

# Build a hash table from arrays.
location_office_data = {}
location_office_data['OFFICE_NAME'] = OFFICE_NAME
location_office_data['NUM_CUSTOMERS'] = NUM_CUSTOMERS
location_office_data['OFFICE_LAT'] = OFFICE_LAT
location_office_data['OFFICE_LON'] = OFFICE_LON


# Build data frame from hash table
df_location_office = pd.DataFrame(data=location_office_data)

In [ ]:
# print the contents of the data frame
df_location_office[:numRows]

### Using folium to display results 

In [ ]:
# folium library intialize base map
def generateBaseMap(default_location=[39.5, -98.35], default_zoom_start=4):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start, tiles="stamentoner")
    return base_map

Clearly an office in Austin or Dallas will reach more customers than one in Las Vegas or San Jose, but still considerably less than in the already existing offices in Atlanta and Pittsburgh.

In the following picture we have visualized this result on a map drawn.

In [ ]:
location_base_map = generateBaseMap()
for index, row in df_location_office.iterrows():
    folium.CircleMarker([row['OFFICE_LAT'], row['OFFICE_LON']],
                        radius=(row['NUM_CUSTOMERS']/700), # in pixel
                        popup=row['OFFICE_NAME'],
                        fill_color="#3db7e4", # divvy color
                       ).add_to(location_base_map)

In [ ]:
location_base_map

From this initial exploration or analysis, a location in Dallas looks most promising. However, is this really the best place? We should probably open up the list of potential future office locations and find out whether there are more suitable locations. For instance, we may want to look at counties with the largest number of high business-volume customers. Since one office might serve multiple counties, the customers in neighboring counties should also be considered.

For comparison, let’s first get the count for the existing and planned offices. To optimize the query runtime we use a two stage approach: in the first pass the st_envIntersects function reduces the candidates. The second pass finally checks which county contains the location:

In [ ]:
# First WITH clause offloc - find id of the counties where the offices are located
# Second WITH clause custcount - count customers in these counties
# Third WITH clause offnei -  find id of the neighboring counties where the offices are
# Fourth WITH clause neighborcount -  count customers in neighboring counties
query=" \
with offloc(officeid, countyid) as \
(select off.id, gc.objectid \
 from offices off, geo_county gc \
 where gc.xmin <= st_x(off.location) and \
       st_x(off.location) <= gc.xmax and \
       gc.ymin <= st_y(off.location) and \
       st_y(off.location) <= gc.ymax and \
       st_envIntersects(gc.shape, off.location) = 1 and \
       st_contains(gc.shape, off.location) = 1 \
), \
custcount (officeid, count_customers) as \
( \
select officeid, count(*) \
from offloc ol, geo_county gc, geo_customer cust \
where ol.countyid = gc.objectid and \
      cust.insurance_value > 200000 and \
      gc.xmin <= cust.xmax and \
      cust.xmin <= gc.xmax and \
      gc.ymin <= cust.ymax and \
      cust.ymin <= gc.ymax and \
      st_envIntersects(gc.shape, cust.shape) = 1 and \
      st_contains(gc.shape, cust.shape) = 1 \
group by officeid \
), \
offnei(officeid, countyid) as \
(select officeid, gnc.objectid \
 from offloc ol, geo_county gc, geo_county gnc \
 where ol.countyid = gc.objectid and \
    gc.objectid <> gnc.objectid and \
    st_touches(gc.shape, gnc.shape) = 1 \
), \
neighborcount (officeid, count_neighbors) as \
( \
select officeid, count(*) \
from offnei on, geo_county gc, geo_customer cust \
where on.countyid = gc.objectid and \
      cust.insurance_value > 200000 and \
      gc.xmin <= cust.xmax and \
      cust.xmin <= gc.xmax and \
      gc.ymin <= cust.ymax and \
      cust.ymin <= gc.ymax and \
      st_envIntersects(gc.shape, cust.shape) = 1 and \
      st_contains(gc.shape, cust.shape) = 1 \
group by officeid \
) \
select cc.officeid, off.name, off.status, cc.count_customers, nc.count_neighbors \
from custcount cc, neighborcount nc, offices off \
where cc.officeid = nc.officeid and off.id = cc.officeid"


stmt = ibm_db.exec_immediate(connection,query)

result = ibm_db.fetch_both(stmt)
OFFICE_ID = []
OFFICE_NAME = []
OFFICE_STATUS = []
NUM_CUSTOMERS = []
NUM_NEIGHBORS =[]
while( result ):
    OFFICE_ID.append(result[0])
    OFFICE_NAME.append(result[1])
    OFFICE_STATUS.append(result[2])
    NUM_CUSTOMERS.append(result[3])
    NUM_NEIGHBORS.append(result[4])
    result = ibm_db.fetch_both(stmt)

In [ ]:
numRows = len(OFFICE_ID)
txt = "Office num results = {}"
print(txt.format(numRows))

# Create data frame
location_data_2 = {}
location_data_2['OFFICE_ID'] = OFFICE_ID
location_data_2['OFFICE_NAME'] = OFFICE_NAME
location_data_2['NUM_CUSTOMERS'] = NUM_CUSTOMERS
location_data_2['NUM_NEIGHBORS'] = NUM_NEIGHBORS
df_location_data_2 = pd.DataFrame(data=location_data_2)

In [ ]:
df_location_data_2[:numRows]

We are now ready to explore other locations and query for the 10 counties with the highest customer count adding the customers in neighboring counties (be patient, this might take a minute to complete):

In [ ]:
# Find top 10 counties with customers

# First WITH clause candidates - get the counties with most customers and their customer count
# Second WITH clause neighbors - get the neighboring counties of top 10 counties
# Third WITH clause neighborcount - count customers in neighboring counties
query=" \
with \
candidates(objectid, name, count_customers) as \
( \
select county.objectid, county.name, count(*) as count_customers \
from geo_customer cust, geo_county county \
where insurance_value > 200000 and \
      county.xmin <= cust.xmax and \
      cust.xmin <= county.xmax and \
      county.ymin <= cust.ymax and \
      cust.ymin <= county.ymax and \
      st_contains(county.shape, cust.shape) = 1 \
group by county.name, county.objectid \
order by count_customers desc \
fetch first 10 rows only \
), \
neighbors(candid, neighbor_id) as \
( \
select cand.objectid, gc2.objectid \
from candidates cand, geo_county gc1, geo_county gc2 \
where cand.objectid = gc1.objectid and \
      gc1.objectid <> gc2.objectid and \
      st_touches(gc1.shape, gc2.shape) = 1 \
), \
neighborcount(candid, count_neighbors) as \
( \
select candid, count(*) \
from neighbors n, geo_county co, geo_customer cust \
where n.neighbor_id = co.objectid and \
          insurance_value > 200000 and \
          co.xmin <= cust.xmax and \
          cust.xmin <= co.xmax and \
          co.ymin <= cust.ymax and \
          cust.ymin <= co.ymax and \
          st_contains(co.shape, cust.shape) = 1 \
group by n.candid \
) \
select c.name, c.objectid, c.count_customers, nc.count_neighbors, st_astext(gc.shape) as wkt \
from candidates c, neighborcount nc, geo_county gc \
where c.objectid = nc.candid and c.objectid = gc.objectid \
order by c.count_customers + nc.count_neighbors desc"

# Takes about 2min30s on SMP on VM.
stmt = ibm_db.exec_immediate(connection,query)

result = ibm_db.fetch_both(stmt)
COUNTY_NAME = []
COUNTY_ID = []
NUM_CUSTOMERS = []
NUM_NEIGHBORS = []
COUNTY_WKT = []
while( result ):
    COUNTY_NAME.append(result[0])
    COUNTY_ID.append(result[1])
    NUM_CUSTOMERS.append(result[2])
    NUM_NEIGHBORS.append(result[3])
    COUNTY_WKT.append(result[4])
    result = ibm_db.fetch_both(stmt)

In [ ]:
numRows = len(COUNTY_NAME)
txt = "Office num results = {}"
print(txt.format(numRows))

# Create data frame.
location_county_data = {}
location_county_data['COUNTY_NAME'] = COUNTY_NAME
location_county_data['COUNTY_ID'] = COUNTY_ID
location_county_data['NUM_CUSTOMERS'] = NUM_CUSTOMERS
location_county_data['NUM_NEIGHBORS'] = NUM_NEIGHBORS
location_county_data['COUNTY_WKT'] = COUNTY_WKT
df_location_county_data = pd.DataFrame(data=location_county_data)
# Convert WKT into geometry type.
df_location_county_data['COUNTY_WKT'] = df_location_county_data['COUNTY_WKT'].apply(wkt.loads)
# Create a GeoDataFrame.
df_gpd_location_county = gpd.GeoDataFrame(df_location_county_data, geometry='COUNTY_WKT')

In [ ]:
# Display the data frame data.
df_location_county_data[:numRows]

Interesting. According to this analysis, a location in county Carroll would be best.

See the following image that displays the results of the query that color-codes the number of customers in the county and its neighboring counties (red means many customers, blue means fewer customers). You can also see the neighboring counties that contribute to the total number of customers.

Build a map showing 10 counties with most customers.

In [ ]:
county_base_map = folium.Map(location=[39.5, -98.35], zoom_start=4, tiles='CartoDB positron')

# Build the county ploygons of the 10 counties.
for index, row in df_gpd_location_county.iterrows():
    # Extract each geometry as GeoSeries and convert to GeoJSON.
    sim_geo = gpd.GeoSeries(row['COUNTY_WKT'])
    geo_j = sim_geo.to_json()
    combined_number = row['NUM_CUSTOMERS'] + row['NUM_NEIGHBORS']
    txt = "combined_number = {}\n".format(combined_number)
    print(txt)
    
    # Crudely using different color codes
    if (combined_number > 1800):
        geo_j = folium.GeoJson(data=geo_j,
                               style_function=lambda x: {'fillColor': '#FE0002'})
    elif (combined_number <= 1800 and combined_number > 1470):
        geo_j = folium.GeoJson(data=geo_j,
                               style_function=lambda x: {'fillColor': '#A1015D'})
    else:
        geo_j = folium.GeoJson(data=geo_j,
                               style_function=lambda x: {'fillColor': '#0302FC'})
        
    txt = "{0}: \n Potential Customers = {1}".format(row['COUNTY_NAME'], str(row['NUM_CUSTOMERS'] + row['NUM_NEIGHBORS']))
    folium.Popup(txt).add_to(geo_j)
    folium.Tooltip(row['COUNTY_NAME']).add_to(geo_j)
    geo_j.add_to(county_base_map)

# Show proposed locations.
for index, row in df_location_office.iterrows():
    folium.CircleMarker([row['OFFICE_LAT'], row['OFFICE_LON']],
                        radius=(row['NUM_CUSTOMERS']/700), # in pixel
                        popup=row['OFFICE_NAME'],
                        fill_color="#3db7e4", # divvy color
                       ).add_to(county_base_map)

In [ ]:
# Display the map with the added geometries.
county_base_map

But let’s also check customers in a specified distance like we did earlier, using the center of the individual county.

In [ ]:
# Look in a 200 mile radius for the top 10 counties and get the centroid of the counties
# uses box filter with 3 degree expansion. 1 degree is about 70 statute miles in length.

query=" \
select a.county_id, a.county_name, a.count_customers, st_y(b.centroid) as lat, st_x(b.centroid) as lon \
from \
  (select cc.objectid as county_id, cc.name as county_name, count(*) as count_customers \
   from geo_county cc, geo_customer cust \
   where \
     cust.insurance_value > 200000 and \
     cc.xmin <= cust.xmax + 3.0 and \
     cust.xmin <= cc.xmax + 3.0 and \
     cc.ymin <= cust.ymax + 3.0 and \
     cust.ymin <= cc.ymax + 3.0 and \
     st_distance(cc.shape, cust.shape, 'STATUTE MILE') < 200 \
   group by cc.objectid, cc.name \
   order by count_customers desc) as a, \
    (select gc.objectid as oid, st_centroid(gc.shape) as centroid from geo_county gc) as b \
 where a.county_id = b.oid and a.county_id in (821, 646, 542, 698, 183, 1964, 327, 1075, 579, 418)"

# Takes about 1min15s on SMP VM.
stmt = ibm_db.exec_immediate(connection,query)

result = ibm_db.fetch_both(stmt)
COUNTY_ID_2 = []
COUNTY_NAME_2 = []
NUM_CUSTOMERS_2 = []
COUNTY_CENTROID_LAT = []
COUNTY_CENTROID_LON = []
while( result ):
    COUNTY_ID_2.append(result[0])
    COUNTY_NAME_2.append(result[1])
    NUM_CUSTOMERS_2.append(result[2])
    COUNTY_CENTROID_LAT.append(result[3])
    COUNTY_CENTROID_LON.append(result[4])
    result = ibm_db.fetch_both(stmt)

In [ ]:
# Create data frame.
location_county_data_2 = {}
location_county_data_2['COUNTY_ID'] = COUNTY_ID_2
location_county_data_2['COUNTY_NAME'] = COUNTY_NAME_2
location_county_data_2['NUM_CUSTOMERS'] = NUM_CUSTOMERS_2
location_county_data_2['COUNTY_CENTROID_LAT'] = COUNTY_CENTROID_LAT
location_county_data_2['COUNTY_CENTROID_LON'] = COUNTY_CENTROID_LON
df_location_county_data_2 = pd.DataFrame(data=location_county_data_2)

In [ ]:
# Display the data frame data.
df_location_county_data_2

In [ ]:
county_base_map_2 = folium.Map(location=[39.5, -98.35], zoom_start=4, tiles='CartoDB positron')
# Show proposed locations.
for index, row in df_location_county_data_2.iterrows():
    folium.CircleMarker([row['COUNTY_CENTROID_LAT'], row['COUNTY_CENTROID_LON']],
                        radius=(row['NUM_CUSTOMERS']/700), # in pixel
                        tooltip=row['COUNTY_NAME'],
                        fill_color="#3db7e4", # divvy color
                       ).add_to(county_base_map_2)

In [ ]:
# Display the map with the added geometries.
county_base_map_2

According to this exploration, with both Lincoln and Douglas county located in Missouri, it seems we should look more closely at places in Missouri for our next planned office location.

# End of scenario